In [1]:
using Ferrite

In [2]:
# Quadrature

dim = 1

ip_fe = Lagrange{dim, RefCube, 1}() # interpolation for the function
ip_geo = Lagrange{dim, RefCube, 1}() # interpolation for the geometry
qr = QuadratureRule{dim, RefCube}(1)

# Specify both function + geometry interpolation
cv = CellScalarValues(qr, ip_fe, ip_geo)

CellScalarValues{1, Float64, RefCube} with 2 shape functions and 1 quadrature points

In [3]:
qr.points

1-element Vector{Vec{1, Float64}}:
 [0.0]

In [4]:
qr.weights

1-element Vector{Float64}:
 2.0

In [5]:
f(x::Number) = sin(x)
f(x::Vec) = sin(x[1])

f (generic function with 2 methods)

In [6]:
nodes =  [Node(Vec((0.0))), Node(Vec((10.0)))]

2-element Vector{Node{1, Float64}}:
 Node{1, Float64}([0.0])
 Node{1, Float64}([10.0])

In [7]:
typeof(((0, 1)))

Tuple{Int64, Int64}

In [8]:
#Cell{dim, 2, 2}(NTuple{2, Int}((1,2)))
cells = [Line((1,2))]

1-element Vector{Line}:
 Line((1, 2))

In [9]:
grid = Grid(cells, nodes)

Grid{1, Line, Float64} with 1 Line cells and 2 nodes

In [10]:
coords = getcoordinates(grid, 1)

2-element Vector{Vec{1, Float64}}:
 [0.0]
 [10.0]

In [11]:
cellIter = CellIterator(grid)

CellIterator{1, Line, Float64, Nothing}(UpdateFlags(true, true, true), Grid{1, Line, Float64}(Line[Line((1, 2))], Node{1, Float64}[Node{1, Float64}([0.0]), Node{1, Float64}([10.0])], Dict{String, Set{Int64}}(), Dict{String, Set{Int64}}(), Dict{String, Set{FaceIndex}}(), Dict{String, Set{EdgeIndex}}(), Dict{String, Set{VertexIndex}}(), sparse(Int64[], Int64[], Bool[], 0, 0)), Ferrite.ScalarWrapper{Int64}(0), [0, 0], Vec{1, Float64}[[0.0], [0.0]], nothing, nothing, #undef)

In [12]:
for cell in CellIterator(grid)
    reinit!(cv, cell)
end

In [13]:
cv.dNdξ

2×1 Matrix{Vec{1, Float64}}:
 [-0.5]
 [0.5]

In [16]:
cv.dNdx

2×1 Matrix{Vec{1, Float64}}:
 [-0.1]
 [0.1]

In [14]:
cell = collect(cellIter)[1]

CellIterator{1, Line, Float64, Nothing}(UpdateFlags(true, true, true), Grid{1, Line, Float64}(Line[Line((1, 2))], Node{1, Float64}[Node{1, Float64}([0.0]), Node{1, Float64}([10.0])], Dict{String, Set{Int64}}(), Dict{String, Set{Int64}}(), Dict{String, Set{FaceIndex}}(), Dict{String, Set{EdgeIndex}}(), Dict{String, Set{VertexIndex}}(), sparse(Int64[], Int64[], Bool[], 0, 0)), Ferrite.ScalarWrapper{Int64}(1), [1, 2], Vec{1, Float64}[[0.0], [10.0]], nothing, nothing, #undef)

In [15]:
cv.detJdV

1-element Vector{Float64}:
 10.0

In [16]:
cv.qr

QuadratureRule{1, RefCube, Float64}([2.0], Vec{1, Float64}[[0.0]])

In [17]:
qr

QuadratureRule{1, RefCube, Float64}([2.0], Vec{1, Float64}[[0.0]])

In [18]:
qr.points

1-element Vector{Vec{1, Float64}}:
 [0.0]

In [19]:
quad_points = getnquadpoints(cv)

1

In [20]:
integration_value = 0.0

for qp in getnquadpoints(cv)
    integration_value += function_value(cv, qp, cell.coords)[1]*getdetJdV(cv, qp)
end

print(integration_value)


50.0

In [21]:
f_nodes = f.(cell.coords)

2-element Vector{Float64}:
  0.0
 -0.5440211108893698

In [22]:
function_gradient(cv, 1, f_nodes)

1-element Vec{1, Float64}:
 -0.05440211108893698

In [23]:
gradient_value = 0.0

for qp in getnquadpoints(cv)
    gradient_value += function_gradient(cv, qp, f_nodes)[1]
end

print(gradient_value)


-0.05440211108893698

In [24]:
cell.coords

2-element Vector{Vec{1, Float64}}:
 [0.0]
 [10.0]

In [25]:
vals_ = values(cellIter)

CellIterator{1, Line, Float64, Nothing}(UpdateFlags(true, true, true), Grid{1, Line, Float64}(Line[Line((1, 2))], Node{1, Float64}[Node{1, Float64}([0.0]), Node{1, Float64}([10.0])], Dict{String, Set{Int64}}(), Dict{String, Set{Int64}}(), Dict{String, Set{FaceIndex}}(), Dict{String, Set{EdgeIndex}}(), Dict{String, Set{VertexIndex}}(), sparse(Int64[], Int64[], Bool[], 0, 0)), Ferrite.ScalarWrapper{Int64}(1), [1, 2], Vec{1, Float64}[[0.0], [10.0]], nothing, nothing, #undef)

In [26]:
cell.nodes

2-element Vector{Int64}:
 1
 2

In [27]:
# Degrees of Freedom
dh = DofHandler(grid)

DofHandler
  Fields:
  Not closed!

In [28]:
# define a new field
push!(dh, :u, 3) # dim=3

DofHandler
  Fields:
    :u, interpolation: Lagrange{1, RefCube, 1}(), dim: 3
  Not closed!

In [33]:
?Ferrite.push!

ErrorException: syntax: invalid identifier name "?"

In [34]:
dof_range(dh, :u)

1:6

In [35]:
Ferrite.getfieldnames(dh)

1-element Vector{Symbol}:
 :u

In [36]:
# Assembly: